In [1]:
import os
WORK = os.environ["WORK"]
PROJ_DIR = f'{WORK}/ADA_Project'
os.chdir(PROJ_DIR)
print(os.getcwd())

import sys
sys.path.insert(0, f'{WORK}/ADA_Project')
sys.path.insert(0, f'{WORK}/ADA_Project/DeepFAMS')

/lustre/work/chaselab/malyetama/ADA_Project


In [2]:
from glob import glob
from tqdm import tqdm
from pathlib import Path
import shutil
from pprint import pprint
from datetime import datetime

import DeepFAMS

Tensorflow version: 1.14.0


In [16]:
def backup_pickles(PROJ_DIR):
    
    Tstamp = datetime.now().strftime("%m%d%Y_%H%M")
    
    WORK = Path(PROJ_DIR).parent
    backups_folder = f'{WORK}/ADA_Project_pkl_backups/{Tstamp}'
    Path(backups_folder).mkdir(parents=True, exist_ok=True)
    
    TRfolders = glob(f'{PROJ_DIR}/training_runs/*')
    
    latest_snaps = []
    
    for folder in TRfolders:
        for num in range(-1, -10, -1):
            files = sorted(glob(folder + "/**/*"))
            files = [x for x in files if 'network-snapshot' in x]
            if files == []:
                continue
            else:
                latest_snap = sorted(files)[-1]
                latest_snaps.append(latest_snap)
                break


    print('=' * 100,
         '\nLatest snaps:\n')
    pprint(latest_snaps)
    print('=' * 100, '\n')

    for pkl in latest_snaps:
        pkl_dst_name = f'{backups_folder}/{Path(pkl).parent.name}__{Path(pkl).name}'
        shutil.copyfile(pkl, pkl_dst_name)
        print(f'Copied: {Path(pkl).parent.parent.name.replace("_training-runs", "")}\'s latest pickle '
             f'to ==> {pkl_dst_name}')

    print('\nUploading to Google Drive...')
    
    DeepFAMS.utils.executePopen(f'''#!/bin/bash
    module load rclone
    rclone copy {backups_folder} GoogleDrive:/ADA_Project_pkl_backups/{Path(backups_folder).name}''',
    PROJ_DIR)

    print('\nDone!')

In [ ]:
backup_pickles(PROJ_DIR)

Latest snaps:

['/work/chaselab/malyetama/ADA_Project/training_runs/AFHQ_training-runs/00001-AFHQ_custom-auto2-resumecustom/network-snapshot-000086.pkl',
 '/work/chaselab/malyetama/ADA_Project/training_runs/metfaces_training-runs/00003-metfaces_custom-auto2-resumecustom/network-snapshot-000225.pkl',
 '/work/chaselab/malyetama/ADA_Project/training_runs/StanfordDogs_training-runs/00002-StanfordDogs_custom-auto2-resumecustom/network-snapshot-000110.pkl',
 '/work/chaselab/malyetama/ADA_Project/training_runs/cars196_training-runs/00012-cars196_custom-auto2-resumecustom/network-snapshot-000245.pkl',
 '/work/chaselab/malyetama/ADA_Project/training_runs/AFHQ-DOG_training-runs/00002-AFHQ-DOG_custom-auto2-resumecustom/network-snapshot-000061.pkl',
 '/work/chaselab/malyetama/ADA_Project/training_runs/102flowers_training-runs/00022-102flowers_custom-auto2-resumecustom/network-snapshot-000245.pkl']

Copied: AFHQ's latest pickle to ==> /work/chaselab/malyetama/ADA_Project_pkl_backups/03192021_1254/0